Attempt to train customized version of SSD for cropping image down to focus the hand.

In [ ]:
import os
import urllib.request

In [ ]:
urllib.request.urlretrieve('http://vision.soic.indiana.edu/egohands_files/egohands_data.zip', 'egohands_data.zip')

('egohands_data.zip', <http.client.HTTPMessage at 0x7f6852bf0f40>)

In [ ]:
import scipy.io as sio
import numpy as np
import os
import gc
import six.moves.urllib as urllib
import cv2
import time
import xml.etree.cElementTree as ET
import random
import shutil as sh
from shutil import copyfile
import zipfile
from google.colab.patches import cv2_imshow
import csv


def save_csv(csv_path, csv_content):
    with open(csv_path, 'w') as csvfile:
        wr = csv.writer(csvfile)
        for i in range(len(csv_content)):
            wr.writerow(csv_content[i])


def get_bbox_visualize(base_path, dir):
    image_path_array = []
    for root, dirs, filenames in os.walk(base_path + dir):
        for f in filenames:
            if(f.split(".")[1] == "jpg"):
                img_path = base_path + dir + "/" + f
                image_path_array.append(img_path)

    #sort image_path_array to ensure its in the low to high order expected in polygon.mat
    image_path_array.sort()
    boxes = sio.loadmat(
        base_path + dir + "/polygons.mat")
    # there are 100 of these per folder in the egohands dataset
    polygons = boxes["polygons"][0]
    # first = polygons[0]
    # print(len(first))
    pointindex = 0

    for first in polygons:
        index = 0

        font = cv2.FONT_HERSHEY_SIMPLEX

        img_id = image_path_array[pointindex]
        img = cv2.imread(img_id)

        img_params = {}
        img_params["width"] = np.size(img, 1)
        img_params["height"] = np.size(img, 0)
        head, tail = os.path.split(img_id)
        img_params["filename"] = tail
        img_params["path"] = os.path.abspath(img_id)
        img_params["type"] = "train"
        pointindex += 1

        boxarray = []
        csvholder = []
        for pointlist in first:
            pst = np.empty((0, 2), int)
            max_x = max_y = min_x = min_y = height = width = 0

            findex = 0
            for point in pointlist:
                if(len(point) == 2):
                    x = int(point[0])
                    y = int(point[1])

                    if(findex == 0):
                        min_x = x
                        min_y = y
                    findex += 1
                    max_x = x if (x > max_x) else max_x
                    min_x = x if (x < min_x) else min_x
                    max_y = y if (y > max_y) else max_y
                    min_y = y if (y < min_y) else min_y
                    # print(index, "====", len(point))
                    appeno = np.array([[x, y]])
                    pst = np.append(pst, appeno, axis=0)
                    cv2.putText(img, ".", (x, y), font, 0.7,
                                (255, 255, 255), 2, cv2.LINE_AA)

            hold = {}
            hold['minx'] = min_x
            hold['miny'] = min_y
            hold['maxx'] = max_x
            hold['maxy'] = max_y
            if (min_x > 0 and min_y > 0 and max_x > 0 and max_y > 0):
                boxarray.append(hold)
                labelrow = [tail,
                            np.size(img, 1), np.size(img, 0), "hand", min_x, min_y, max_x, max_y]
                csvholder.append(labelrow)

            cv2.polylines(img, [pst], True, (0, 255, 255), 1)
            cv2.rectangle(img, (min_x, max_y),
                          (max_x, min_y), (0, 255, 0), 1)

        csv_path = img_id.split(".")[0]
        if not os.path.exists(csv_path + ".csv"):
            cv2.putText(img, "DIR : " + dir + " - " + tail, (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (77, 255, 9), 2)
            #cv2_imshow('Verifying annotation ', img)
            save_csv(csv_path + ".csv", csvholder)
            #print("===== saving csv file for ", tail)
        cv2.waitKey(2)  # close window when a key press is detected


def create_directory(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# combine all individual csv files for each image into a single csv file per folder.


def generate_label_files(image_dir):
    header = ['filename', 'width', 'height',
              'class', 'xmin', 'ymin', 'xmax', 'ymax']
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            csvholder = []
            csvholder.append(header)
            loop_index = 0
            for f in os.listdir(image_dir + dir):
                if(f.split(".")[1] == "csv"):
                    loop_index += 1
                    #print(loop_index, f)
                    csv_file = open(image_dir + dir + "/" + f, 'r')
                    reader = csv.reader(csv_file)
                    for row in reader:
                        csvholder.append(row)
                    csv_file.close()
                    os.remove(image_dir + dir + "/" + f)
            save_csv(image_dir + dir + "/" + dir + "_labels.csv", csvholder)
            print("Saved label csv for ", dir, image_dir +
                  dir + "/" + dir + "_labels.csv")


# Split data, copy to train/test folders
def split_data_test_eval_train(image_dir):
    create_directory("images")
    create_directory("images/train")
    create_directory("images/test")

    data_size = 4000
    loop_index = 0
    data_sampsize = int(0.1 * data_size)
    test_samp_array = random.sample(range(data_size), k=data_sampsize)

    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            for f in os.listdir(image_dir + dir):
                if(f.split(".")[1] == "jpg"):
                    loop_index += 1
                    print(loop_index, f)

                    if loop_index in test_samp_array:
                        os.rename(image_dir + dir +
                                  "/" + f, "images/test/" + f)
                        os.rename(image_dir + dir +
                                  "/" + f.split(".")[0] + ".csv", "images/test/" + f.split(".")[0] + ".csv")
                    else:
                        os.rename(image_dir + dir +
                                  "/" + f, "images/train/" + f)
                        os.rename(image_dir + dir +
                                  "/" + f.split(".")[0] + ".csv", "images/train/" + f.split(".")[0] + ".csv")
                    print(loop_index, image_dir + f)
            print(">   done scanning director ", dir)
            os.remove(image_dir + dir + "/polygons.mat")
            os.rmdir(image_dir + dir)

        print("Train/test content generation complete!")
        generate_label_files("images/")


def generate_csv_files(image_dir):
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            get_bbox_visualize(image_dir, dir)

    print("CSV generation complete!\nGenerating train/test/eval folders")
    split_data_test_eval_train("egohands/_LABELLED_SAMPLES/")


# rename image files so we can have them all in a train/test/eval folder.
def rename_files(image_dir):
    print("Renaming files")
    loop_index = 0
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            for f in os.listdir(image_dir + dir):
                if (dir not in f):
                    if(f.split(".")[1] == "jpg"):
                        loop_index += 1
                        os.rename(image_dir + dir +
                                  "/" + f, image_dir + dir +
                                  "/" + dir + "_" + f)
                else:
                    break

    generate_csv_files("egohands/_LABELLED_SAMPLES/")

def extract_folder(dataset_path):
    print("Egohands dataset already downloaded.\nGenerating CSV files")
    if not os.path.exists("egohands"):
        zip_ref = zipfile.ZipFile(dataset_path, 'r')
        print("> Extracting Dataset files")
        zip_ref.extractall("egohands")
        print("> Extraction complete")
        zip_ref.close()
        rename_files("egohands/_LABELLED_SAMPLES/")

def download_egohands_dataset(dataset_url, dataset_path):
    is_downloaded = os.path.exists(dataset_path)
    if not is_downloaded:
        print(
            "> downloading egohands dataset. This may take a while (1.3GB, say 3-5mins). Coffee break?")
        opener = urllib.request.URLopener()
        opener.retrieve(dataset_url, dataset_path)
        print("> download complete")
        extract_folder(dataset_path);

    else:
        extract_folder(dataset_path)


EGOHANDS_DATASET_URL = "http://vision.soic.indiana.edu/egohands_files/egohands_data.zip"
EGO_HANDS_FILE = "egohands_data.zip"


download_egohands_dataset(EGOHANDS_DATASET_URL, EGO_HANDS_FILE)

In [ ]:
!pip install object-detection

In [ ]:
%cd /content
!python generate_tfrecord.py --csv_input=/content/images/train/train_labels.csv --output_path=train.record --image_dir=/content/images/train
!python generate_tfrecord.py --csv_input=/content/images/test/test_labels.csv --output_path=test.record --image_dir=/content/images/test

In [ ]:
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2023-11-08 18:53:11--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.200.207, 74.125.130.207, 74.125.68.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.200.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  9.90MB/s    in 2.0s    

2023-11-08 18:53:13 (9.90 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [ ]:
!mkdir /content/mainmodel
!mv /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz /content/mainmodel
!tar -xf /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -C /content/mainmodel/
!mv -f /content/pipeline.config /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
# !mv /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint0

In [ ]:
%cd /content
!git clone https://github.com/tensorflow/models.git

/content
Cloning into 'models'...
remote: Enumerating objects: 89661, done.
remote: Counting objects: 100% (3369/3369), done.
remote: Compressing objects: 100% (1813/1813), done.
remote: Total 89661 (delta 1752), reused 3081 (delta 1526), pack-reused 86292
Receiving objects: 100% (89661/89661), 605.15 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (63663/63663), done.


In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=. --proto_path=/content/models/research/
!cp -r -T /content/models/research/object_detection /usr/local/lib/python3.10/dist-packages/object_detection
!mv -f /content/tfexample_decoder.py /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py

/content/models/research


In [ ]:
!pip install tensorflow-io
!pip install tf-models-official
!pip install lvis
# !pip install tensorflow==2.13.0

In [ ]:
!python3 /content/models/research/slim/setup.py build
!python3 /content/models/research/slim/setup.py install

In [ ]:
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -rf /content/drive/MyDrive/obj_detection_training/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint

In [ ]:
!cp /content/drive/MyDrive/obj_detection_training/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/ckpt-386.data-00000-of-00001 /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-386.data-00000-of-00001
!cp /content/drive/MyDrive/obj_detection_training/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/ckpt-386.index /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-386.index

In [ ]:
!rm /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-361.data-00000-of-00001
!rm /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-361.index

In [ ]:
# !python3 /content/models/research/object_detection/legacy/train.py --logtostderr --train_dir=/content/images/ --pipeline_config_path=/content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
!python3 /content/models/research/object_detection/model_main_tf2.py --alsologtostderr --model_dir=/content/drive/MyDrive/obj_detection_training/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/ --pipeline_config_path=/content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --num_train_steps=200000 --checkpoint_every_n=500 #--use_tpu=true

2023-11-08 18:56:03.273511: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 18:56:03.273561: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 18:56:03.273604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 18:56:03.284253: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 18:56:04.952016: W tensorflow/c

In [ ]:
!rm -r /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model

In [ ]:
!python3 /content/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/obj_detection_training/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/ --pipeline_config_path=/content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --checkpoint_dir=/content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint

2023-11-08 20:34:59.440459: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 20:34:59.440503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 20:34:59.440541: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 20:34:59.448980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 20:35:01.588477: W tensorflow/c

In [ ]:
!python3 /content/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
    --trained_checkpoint_dir /content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint \
    --output_directory hand_inference_graph

In [ ]:
!zip -r hand_inference_graph.zip hand_inference_graph/

In [ ]:
!pip install roboflow
!mkdir /content/datasets
%cd /content/datasets
from roboflow import Roboflow
rf = Roboflow(api_key="8O0d3piVDubPgWzft4ha")

project = rf.workspace("bdbi").project("image-annotation-lxsso")
dataset = project.version(5).download("yolov8")

In [ ]:
%cd /content

/content


In [ ]:
# from utils import detector_utils as detector_utils
import cv2
import tensorflow as tf
import datetime
import argparse
import numpy as np
import time
import dataset_gen
from object_detection.utils import config_util
from object_detection.builders import model_builder
import os
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
from tensorflow import squeeze

# detection_graph, sess = detector_utils.load_inference_graph()
# Loading saved mode.
# model = tf.saved_model.load("hand_inference_graph/saved_model")
PATH_TO_MODEL_DIR = "/content/mainmodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"
CKPT_NUM = 386

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, f'ckpt-{CKPT_NUM}')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

from google.colab.patches import cv2_imshow
path_to_train_images = '/content/datasets/Image-Annotation-5/train/images'
# path_to_train_labels = '/content/Image-Annotation-5/train/labels'
path_to_test_images = '/content/datasets/Image-Annotation-5/test/images'
# path_to_test_labels = '/content/Image-Annotation-5/test/labels'
path_to_val_images = '/content/datasets/Image-Annotation-5/valid/images'
# path_to_val_labels = '/content/Image-Annotation-5/valid/labels'
category_index = {0: {'id': 1, 'name': 'hand'}}

# def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
#     # initialize the dimensions of the image to be resized and
#     # grab the image size
#     dim = None
#     (h, w) = image.shape[:2]
#     if h>w:


#     # if both the width and height are None, then return the
#     # original image
#     if width is None and height is None:
#         return image

#     # check to see if the width is None
#     if width is None:
#         # calculate the ratio of the height and construct the
#         # dimensions
#         r = height / float(h)
#         dim = (int(w * r), height)

#     # otherwise, the height is None
#     else:
#         # calculate the ratio of the width and construct the
#         # dimensions
#         r = width / float(w)
#         dim = (width, int(h * r))

#     # resize the image
#     resized = cv2.resize(image, dim, interpolation = inter)

#     # return the resized image
#     return resized


if __name__ == '__main__':

    # parser = argparse.ArgumentParser()
    # parser.add_argument(
    #     '-sth',
    #     '--scorethreshold',
    #     dest='score_thresh',
    #     type=float,
    #     default=0.2,
    #     help='Score threshold for displaying bounding boxes')

    # args = parser.parse_args()

    start_time = datetime.datetime.now()
    num_frames = 0
    im_width, im_height = (512, 513)
    # max number of hands we want to detect/track
    num_hands_detect = 1

    #cv2.namedWindow('Single-Threaded Detection', cv2.WINDOW_NORMAL)

    for image_path in dataset_gen.generate_stream(path_to_val_images):
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]

        # Pre-processing image.
        image1 = tf.image.decode_image(open(image_path, 'rb').read(), channels=3)
        image = tf.expand_dims(image1, 0)
        image = tf.image.resize_with_pad(image, 640, 640)
        # image_np = cv2.imread(image_path)
        # image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
        detections = detect_fn(image)
        bbox = detections['detection_boxes'][0][0]
        bclass = detections['detection_classes'][0][0]
        bscore = detections['detection_scores'][0][0]

        print(bbox, bclass, bscore)

        finalbox = []
        class_label = None
        image_display = None
        if bscore >= 0.2:
            y_min = int(bbox[0] * im_height)
            x_min = int(bbox[1] * im_width)
            y_max = int(bbox[2] * im_height)
            x_max = int(bbox[3] * im_width)
            class_label = category_index[int(bclass)]['name']
            finalbox = [x_min, y_min, x_max, y_max, class_label, float(bscore)]
        # return (bbox, class_labels)

        # det_boxes, class_labels = ExtractBBoxes(bboxes, bclasses, bscores, im_width, im_height)
            print(finalbox)
            print(class_label)


            # image_np = cv2.flip(image_np, 1)
            # try:
            #     image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            # except:
            #     print("Error converting to RGB")

            # Actual detection. Variable boxes contains the bounding box cordinates for hands detected,
            # while scores contains the confidence for each of these boxes.
            # Hint: If len(boxes) > 1 , you may assume you have found atleast one hand (within your score threshold)

            # # draw bounding boxes on frame
            # detector_utils.draw_box_on_image(num_hands_detect, args.score_thresh,
            #                                  scores, boxes, im_width, im_height,
            #                                  image_np, 0.75)

            p1x = p2x = p1y = p2y = None
            height = width = None
            p1xbox = p2xbox = p1ybox = p2ybox = None
            (left, right, top, bottom) = (finalbox[0], finalbox[2], finalbox[1], finalbox[3])
            padding_percent = 0.75
            p1x = max(0, int(left) - int(left*padding_percent//2))
            p1y = max(0, int(top) - int(top*padding_percent//2))
            p2x = min(640, int(right) + int(right*padding_percent//2))
            p2y = min(640, int(bottom) + int(bottom*padding_percent//2))

            height = p2y - p1y
            width = p2x - p1x

            p1xbox = int(left*padding_percent//2)
            p2xbox = width - int(right*padding_percent//2)
            p1ybox = int(top*padding_percent//2)
            p2ybox = height - int(bottom*padding_percent//2)

            # print(squeeze(image).numpy().astype(np.uint8))
            image_np = cv2.cvtColor(tf.image.resize_with_pad(image1, 640, 640).numpy(), cv2.COLOR_RGB2BGR)

            #cv2_imshow(image_np)

            # (p1x, p1y, p2x, p2y) = (p1xbox, p1ybox, p2xbox, p2ybox)
            image_display = image_np[p1y:p2y, p1x:p2x]
        else:
            image_display = cv2.imread(image_path)

        # (p1x, p1y, p2x, p2y), image_display = detector_utils.get_image_cropped(num_hands_detect, args.score_thresh,
        #                                  scores, boxes, im_width, im_height,
        #                                  image_np, 0.75)
        # image_display = cv2.cvtColor(image_display, cv2.COLOR_RGB2BGR)
        # cv2.rectangle(image_display, (p1x, p1y), (p2x, p2y), (77, 255, 9), 3, 1)

        # Calculate Frames per second (FPS)
        num_frames += 1
        elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
        fps = num_frames / elapsed_time

        if (finalbox is not None):
            # cv2.drawContours(image_np, contours=(77, 255, 9))
            # image_display = cv2.cvtColor(image_display, cv2.RGB2BGR)
            cv2_imshow(image_display)

            # if cv2.waitKey(25) & 0xFF == ord('q'):
            #     cv2.destroyAllWindows()
            #     break

            dataset_gen.saveImage(image_display, '/content/Image-Cropped/valid/images', image_path.split('/')[-1])
        else:
            print("frames processed: ", num_frames, "elapsed time: ",
                  elapsed_time, "fps: ", str(int(fps)))
